In [27]:
import numpy as np
from scipy import concatenate
import json
import pandas as pd
from collections import Counter
pd.options.display.max_columns = 100

In [28]:
data_2014 = pd.read_csv("merged_shots/merged_shots_14.csv")

In [29]:
data_2013 = pd.read_csv("merged_shots/merged_shots_13.csv")

In [30]:
frames = [data_2013,data_2014]
combined = pd.concat(frames)

combined["unique"] = range(0,len(combined))
for i, c in enumerate(combined.isnull().sum()/combined.shape[0]):
    if c != 0:
        print combined.columns[i], c

HOMEDESCRIPTION 0.471027036451
NEUTRALDESCRIPTION 1.0
PLAYER2_NAME 0.735540915827
PLAYER2_TEAM_ABBREVIATION 0.735540915827
PLAYER2_TEAM_CITY 0.735540915827
PLAYER2_TEAM_ID 0.735540915827
PLAYER2_TEAM_NICKNAME 0.735540915827
PLAYER3_NAME 0.943737780047
PLAYER3_TEAM_ABBREVIATION 0.943737780047
PLAYER3_TEAM_CITY 0.943737780047
PLAYER3_TEAM_ID 0.943737780047
PLAYER3_TEAM_NICKNAME 0.943737780047
SCORE 0.548056638314
VISITORDESCRIPTION 0.472710743596
HOME_TEAM_ID 0.0979165369432
AWAY_TEAM_ID 0.0979165369432
HOME_PLAYER1_ID 0.0979165369432
HOME_PLAYER2_ID 0.0979165369432
HOME_PLAYER3_ID 0.0979165369432
HOME_PLAYER4_ID 0.0979165369432
HOME_PLAYER5_ID 0.0979165369432
AWAY_PLAYER1_ID 0.0979165369432
AWAY_PLAYER2_ID 0.0979165369432
AWAY_PLAYER3_ID 0.0979165369432
AWAY_PLAYER4_ID 0.0979165369432
AWAY_PLAYER5_ID 0.0979165369432
SHOT_CLOCK 0.0519234361574
off_height 0.0205556731715
def_height 0.0239355409158


In [31]:
a = np.concatenate((combined.PLAYER1_NAME, combined.PLAYER2_NAME))
b = np.concatenate((combined.PLAYER1_ID, combined.PLAYER2_ID))

In [32]:
player_id_dict = {k : v for k, v in zip(b, a)}
team_id_dict = {int(k) : v1+" "+v2 for k, v1, v2 in zip(combined.PLAYER1_TEAM_ID, combined.PLAYER1_TEAM_CITY, combined.PLAYER1_TEAM_NICKNAME)}

In [33]:
defender_id_dict = {int(k) : v for k, v in zip(combined.CLOSEST_DEFENDER_PLAYER_ID, combined.CLOSEST_DEFENDER)}

In [34]:
for p in defender_id_dict.keys():
    temp = p in player_id_dict.keys()
    if not temp:
        splt = defender_id_dict[p].split(",")
        player_id_dict[p] = splt[1]+" "+splt[0]

In [35]:
for i, c in enumerate(combined.columns):
        print i, ":", c

0 : Unnamed: 0
1 : EVENTMSGACTIONTYPE
2 : EVENTMSGTYPE
3 : EVENTNUM
4 : GAME_ID
5 : HOMEDESCRIPTION
6 : NEUTRALDESCRIPTION
7 : PCTIMESTRING
8 : PERIOD_x
9 : PERSON1TYPE
10 : PERSON2TYPE
11 : PERSON3TYPE
12 : PLAYER1_ID
13 : PLAYER1_NAME
14 : PLAYER1_TEAM_ABBREVIATION
15 : PLAYER1_TEAM_CITY
16 : PLAYER1_TEAM_ID
17 : PLAYER1_TEAM_NICKNAME
18 : PLAYER2_ID
19 : PLAYER2_NAME
20 : PLAYER2_TEAM_ABBREVIATION
21 : PLAYER2_TEAM_CITY
22 : PLAYER2_TEAM_ID
23 : PLAYER2_TEAM_NICKNAME
24 : PLAYER3_ID
25 : PLAYER3_NAME
26 : PLAYER3_TEAM_ABBREVIATION
27 : PLAYER3_TEAM_CITY
28 : PLAYER3_TEAM_ID
29 : PLAYER3_TEAM_NICKNAME
30 : SCORE
31 : SCOREMARGIN
32 : VISITORDESCRIPTION
33 : WCTIMESTRING
34 : HOME_TEAM_ID
35 : AWAY_TEAM_ID
36 : HOME_PLAYER1_ID
37 : HOME_PLAYER2_ID
38 : HOME_PLAYER3_ID
39 : HOME_PLAYER4_ID
40 : HOME_PLAYER5_ID
41 : AWAY_PLAYER1_ID
42 : AWAY_PLAYER2_ID
43 : AWAY_PLAYER3_ID
44 : AWAY_PLAYER4_ID
45 : AWAY_PLAYER5_ID
46 : ACTION_BEFORE
47 : Unnamed: 0.1
48 : CLOSEST_DEFENDER
49 : CLOSEST_D

In [42]:
combined_ = combined.drop(combined.columns[[0,1,2,4,5,6,7,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,32,33,34,35,36,37,38,39,40,41,42,43,44,45,47,48,49,52,53,54,55,57,58,59,60,63,65,67,71,68,76,72,73,78,79,80,81,85,89]], axis=1)

In [43]:
for i, c in enumerate(combined_.columns):
        print i, ":", c

0 : EVENTNUM
1 : PERIOD_x
2 : SCOREMARGIN
3 : ACTION_BEFORE
4 : CLOSE_DEF_DIST
5 : DRIBBLES
6 : LOCATION
7 : PTS_TYPE
8 : SHOT_CLOCK
9 : SHOT_NUMBER
10 : TOUCH_TIME
11 : GAME_EVENT_ID
12 : PLAYER_ID
13 : MINUTES_REMAINING
14 : SECONDS_REMAINING
15 : ACTION_TYPE
16 : SHOT_DISTANCE
17 : LOC_X
18 : LOC_Y
19 : SHOT_MADE_FLAG
20 : off_height
21 : def_height


In [44]:
def height(thing):
    try:
        f, i = thing.split('-')
    except AttributeError:
        return None
    except ValueError:
        return None
    else:
        return int(f)*12+int(i)

In [45]:
#combined_.W = combined_.W == "W"
combined_['SEC_LEFT_Q'] = combined_.MINUTES_REMAINING*60+combined_.SECONDS_REMAINING
combined_['SEC_LEFT_G'] = np.where(combined_.PERIOD_x < 5, combined_.SEC_LEFT_Q+720*(4-combined_.PERIOD_x), combined_.SEC_LEFT_Q)
combined_.LOCATION = np.where(combined_.LOCATION == "H", 1, 0)
combined_['OFF_HEIGHT'] = combined.off_height.apply(lambda x: height(x))
combined_['DEF_HEIGHT'] = combined.def_height.apply(lambda x: height(x))
combined_['HEIGHT_DIFF'] = combined_['OFF_HEIGHT'] - combined_['DEF_HEIGHT']
combined_['JUMP_SHOT']=[1 if x=='Jump Shot' else 0 for x in combined_.ACTION_TYPE]
combined_['SHOT_DISTxCLOSE_DEF_DIST'] = combined_['SHOT_DISTANCE']*combined_['CLOSE_DEF_DIST']
combined_['HEIGHTxCLOSE_DEF_DIST'] = combined_['HEIGHT_DIFF']*combined_['CLOSE_DEF_DIST']
combined_['SHOT_DISTxJUMP_SHOT']= combined_['SHOT_DISTANCE']*combined_['JUMP_SHOT']
del combined_['MINUTES_REMAINING']
del combined_['SECONDS_REMAINING']
del combined_['off_height']
del combined_['def_height']
del combined_['ACTION_TYPE']

In [46]:
combined_.reset_index(drop=True, inplace=True)

In [49]:
combined_.head()

,EVENTNUM,PERIOD_x,SCOREMARGIN,ACTION_BEFORE,CLOSE_DEF_DIST,DRIBBLES,LOCATION,PTS_TYPE,SHOT_CLOCK,SHOT_NUMBER,TOUCH_TIME,GAME_EVENT_ID,PLAYER_ID,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_MADE_FLAG,SEC_LEFT_Q,SEC_LEFT_G,OFF_HEIGHT,DEF_HEIGHT,HEIGHT_DIFF,JUMP_SHOT,SHOT_DISTxCLOSE_DEF_DIST,HEIGHTxCLOSE_DEF_DIST,SHOT_DISTxJUMP_SHOT
0,2,1,0,10,1.6,2,1,2,6.7,1,7.8,2,2561,5,-38,45,0,699,2859,81,79,2,1,8.0,3.2,5
1,5,1,2,6,4.1,1,1,2,16.8,1,3.2,5,202331,19,105,164,1,690,2850,81,81,0,0,77.9,0.0,0
2,6,1,-2,1,4.4,0,0,3,13.0,1,0.7,6,201167,27,51,266,0,675,2835,77,81,-4,1,118.8,-17.6,27
3,8,1,2,4,0.4,0,1,2,13.8,2,1.8,8,2561,2,28,-5,0,663,2823,81,79,2,0,0.8,0.8,0
4,11,1,5,5,5.7,0,1,3,15.6,1,0.7,11,202362,26,15,260,1,651,2811,77,72,5,1,148.2,28.5,26


In [54]:
combined_.to_csv('parsed_df.csv')

SyntaxError: invalid syntax (<ipython-input-54-a5bb9349d3fd>, line 1)

In [16]:
player_id_dict = {k : v for k, v in zip(b, a)}
team_id_dict = {int(k) : v1+" "+v2 for k, v1, v2 in zip(combined.PLAYER1_TEAM_ID, combined.PLAYER1_TEAM_CITY, combined.PLAYER1_TEAM_NICKNAME)}

In [53]:
pd.DataFrame([[k,v] for k, v in player_id_dict.iteritems()], columns=["ID", "NAME"]).to_json('Player_ID_Dict.json')
pd.DataFrame([[k,v] for k, v in team_id_dict.iteritems()], columns=["ID", "NAME"]).to_json('Team_ID_Dict.json')